In [121]:
import pandas as pd
from typing import Callable, List, Any

In [122]:
test = pd.read_csv('../test.csv')

In [124]:
def get_teammates_average(df: pd.DataFrame, question: str):
    def apply_helper(row, apply_df):
        team_number, teammate_number = row['TeamName'], int(row['TeammateNumber'])
        apply_df = apply_df.loc[
            (apply_df['TeamName'] == team_number) & (apply_df['TeammateNumber'] != teammate_number),
            f'{question}.{teammate_number}'
        ]
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            return np.nanmean(apply_df)

    return df.apply(apply_helper, axis=1, apply_df=df)

In [106]:
def get_response_by_teammate_number(df: pd.DataFrame, teammate_number: int, question: str):
    if f'{question}.{teammate_number}' not in df.columns:
        return df.loc[df['TeammateNumber'] == teammate_number, question]

    team_sizes = df['TeamName'].value_counts()
    emails = df['FullNameLast']
    df = df.loc[df['TeammateNumber'] == teammate_number, df.columns.str.contains(f'^{question}|Team', regex=True)]

    result = [[], []]
    seen_indexes = set()
    for index, row in df.iterrows():
        team_size = team_sizes[row['TeamName']]


        for i in range(1, team_size+1):
            # if f'{question}.{i}' not in df.columns:
            #     break
            if index + i - teammate_number in seen_indexes:
                print(index, i, teammate_number)
            seen_indexes.add(index + i - teammate_number)
            result[0].append(index + i - teammate_number)
            result[1].append(row[f'{question}.{i}'])


    result_series = pd.Series(result[1])
    result_series.index = result[0]
    result_df = result_series.to_frame()
    # print(result_df.index.value_counts())
    # print(result_df.loc[151, :])

    result_df = result_df.join(emails).set_index('FullNameLast')

    return result_df
    
    # return df.loc[:, f'{question}.{teammate_number}']

In [120]:
team_size = max(test['TeammateNumber'].unique())
for i in range(1, team_size+1):
    test_one = ColumnBuilder(
        'DidWell',
        str(i),
        get_response_by_teammate_number,
        [i] + ['14']
    )(test_one)

    print(len(test_one))

197
197
197
197
197
197


In [109]:
from ColumnBuilder import *

In [110]:
test_cb_one = ColumnBuilder('Improve', '3', get_response_by_teammate_number, ['1', '14'])

In [111]:
test_one = test_cb_one(test)

In [112]:
test_one['FullNameLast'].value_counts()

FullNameLast
Ruiz, Daniel              1
Blanset, Kyle             1
Tran, Tam                 1
Almaguer, Matteo          1
Lee, Abraham              1
                         ..
Ho, Jenson                1
Ferweda, Cameron          1
Boecherer, Aaron          1
Herrera, David Antonio    1
Petty, Evan               1
Name: count, Length: 197, dtype: int64

In [113]:
test_cb = PerMemberFunc('Improve', 'PerMember', None, ['14'])

In [114]:
test_result = test_cb(test)

In [115]:
test_result['FullNameLast']

0          Ruiz, Daniel
1       Ortega, Xochitl
2        Hassani, Shadi
3        Chow, Michelle
4        Peraza, Jazlyn
             ...       
192      Gotesky, James
193     Feingold, Jacob
194     Yamaguchi, Eric
195    Libermann, Henry
196         Petty, Evan
Name: FullNameLast, Length: 197, dtype: object